In [ ]:
from triqs.gf import *
from h5 import *
from triqs.plot.mpl_interface import *
import numpy as np
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['savefig.dpi']=100
plt.rcParams['figure.figsize'] = (8.0,6.0)

Solution of the exercises
=========================

Solution of exercise 1
----------------------

The script below solves the two-orbital Hubbard model for a variety of filling, $U$ and $J$.

In [ ]:
# %load scripts/two_band.py
from triqs.gf import *
from triqs.operators import *
from h5 import *
from triqs_cthyb import Solver
from itertools import product
import numpy as np

import os
if not os.path.exists('data/two_bands'):
    os.makedirs('data/two_bands')

# Parameters of the model
t = 1.0
beta = 10.0
n_loops = 10
filling = 'half' # or 'quarter'
n_orbitals = 2

# Construct the solver
S = Solver(beta = beta, gf_struct = [('up-0',1), ('up-1',1), ('down-0',1), ('down-1',1)] )

for coeff in [0.0, 0.1, 0.2]:

    # Run for several values of U
    for U in np.arange(1.0, 13.0, 1.0):

        J = coeff * U

        # Expression of mu for half and quarter filling
        if filling == 'half':
            mu = 0.5*U + 0.5*(U-2*J) + 0.5*(U-3*J)
        elif filling == 'quarter':
            mu = -0.81 + (0.6899-1.1099*coeff)*U + (-0.02548+0.02709*coeff-0.1606*coeff**2)*U**2

        # Set the interacting Kanamori hamiltonian
        h_int = Operator()
        for o in range(0,n_orbitals):
            h_int += U*n('up-%s'%o,0)*n('down-%s'%o,0)
        for o1,o2 in product(list(range(0,n_orbitals)),list(range(0,n_orbitals))):
            if o1==o2: continue
            h_int += (U-2*J)*n('up-%s'%o1,0)*n('down-%s'%o2,0)
        for o1,o2 in product(list(range(0,n_orbitals)),list(range(0,n_orbitals))):
            if o2>=o1: continue;
            h_int += (U-3*J)*n('up-%s'%o1,0)*n('up-%s'%o2,0)
            h_int += (U-3*J)*n('down-%s'%o1,0)*n('down-%s'%o2,0)
        for o1,o2 in product(list(range(0,n_orbitals)),list(range(0,n_orbitals))):
            if o1==o2: continue
            h_int += -J*c_dag('up-%s'%o1,0)*c_dag('down-%s'%o1,0)*c('up-%s'%o2,0)*c('down-%s'%o2,0)
            h_int += -J*c_dag('up-%s'%o1,0)*c_dag('down-%s'%o2,0)*c('up-%s'%o2,0)*c('down-%s'%o1,0)

        # This is a first guess for G
        S.G0_iw << inverse(iOmega_n + mu - t**2 * SemiCircular(2*t))

        # DMFT loop with self-consistency
        for i in range(n_loops):

            print("\n\nIteration = %i / %i" % (i+1, n_loops))

            # Symmetrize the Green's function and use self-consistency
            if i > 0:
                g = 0.25 * ( S.G_iw['up-0'] + S.G_iw['up-1'] + S.G_iw['down-0'] + S.G_iw['down-1'] )
                for name, g0 in S.G0_iw:
                    g0 << inverse(iOmega_n + mu - t**2 * g)

            # Solve the impurity problem
            S.solve(h_int = h_int,
                    n_cycles  = 30000,
                    length_cycle = 100,
                    n_warmup_cycles = 5000)

            # Check density
            for name, g in S.G_iw:
                print(name, ": ",g.density()[0,0].real)

            # Save iteration in archive
            with HDFArchive("data/two_bands/%s-U%.2f-J%.2f.h5"%(filling,U,J)) as A:
                A['G-%i'%i] = S.G_iw
                A['Sigma-%i'%i] = S.Sigma_iw


Solution of exercise 2
----------------------

The solution of the exercise is in the script called `run_two_bands.py` in the tutorial directory.
The script was run from the shell to see the Monte Carlo progress. As you can see in the script, the Green's functions and self-energies are saved in archives. These archives are in the `data/two_bands` subdirectory. Here is a plot of
the Green's functions for different values of $U$ at given $J$'s.

### J = 0.0

You will see that the Mott transition is somewhere between 6.0 and 7.0.

In [ ]:
coeff = 0.0
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/half-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### J = 0.1 U

You will see that the Mott transition is somewhere between 4.0 and 5.0.

In [ ]:
coeff = 0.1
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/half-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### J = 0.2 U

You will see that the Mott transition is somewhere between 3.0 and 4.0.

In [ ]:
coeff = 0.2
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/half-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### Conclusion of Exercise 2

The value of $U_c$ is decreasing with increasing values of $J$! Can you understand why?

Solution of exercise 3
----------------------

The solution of the exercise is again the script called `run_two_bands.py` in the tutorial directory.
You will have to change `filling = 'quarter'`. The generated archives are in the `results` subdirectory. Here is a plot of
the Green's functions for different values of $U$ at given $J$'s.

### J = 0.0

You will se that the Mott transition is somewhere between 5.0 and 6.0.

In [ ]:
coeff = 0.0
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/quarter-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### J = 0.1 U

You will see that the Mott transition is somewhere between 7.0 and 8.0.

In [ ]:
coeff = 0.1
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/quarter-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### J = 0.2 U

You will see that the Mott transition happens at values of $U$ larger than 12.

In [ ]:
coeff = 0.2
for U in np.arange(1.0, 13.0):

    J = coeff * U
    A = HDFArchive("data/two_band/quarter-U%.2f-J%.2f.h5"%(U,J), 'r')
    oplot(A['G-9']['up-0'].imag, 'o', name="U = %.2f"%U)

plt.xlim(0,10)

### Conclusion of Exercise 3

Now the value of $U_c$ is increasing with increasing values of $J$! Why?